In [12]:
import os
google_api_key = os.getenv("MY_GOOGLE_AI_STUDIO_KEY")  # Fetch secret from env variable

In [ ]:
# Write code to get a gemini model
import google.generativeai as genai
genai.configure(api_key=google_api_key)
model = genai.GenerativeModel('gemini-2.0-flash-exp')
response = model.generate_content("Explain how AI works")
print(response.text)

In [ ]:
def get_files_in_folder(root_dir):    
    """Recursively scan a folder and return list of files with full paths"""
    import os
    file_list = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            full_path = os.path.join(dirpath, filename)
            file_list.append(full_path)
    return file_list

In [ ]:
def get_file_types(file_list):    
    """Return dictionary mapping filenames to their file extensions."""
    import os
    type_dict = {}
    for file_path in file_list:
        # Split the file extension from the path
        _, ext = os.path.splitext(file_path)
        # Remove leading dot and convert to lowercase for consistency
        clean_ext = ext.lower().lstrip('.') if ext else 'no_extension'
        type_dict[file_path] = clean_ext
    return type_dict

In [1]:
def organize_files(type_dict):
    """Organize files into folders based on their type in their respective directories."""
    import os
    import shutil
    
    for file_path, file_type in type_dict.items():
        # Get parent directory and filename
        parent_dir = os.path.dirname(file_path)
        filename = os.path.basename(file_path)
        
        # Create target directory name (pluralize for readability)
        target_dir_name = f"{file_type}s"  # Example: 'pdf' becomes 'pdfs'
        target_dir = os.path.join(parent_dir, target_dir_name)
        
        # Create target directory if it doesn't exist
        os.makedirs(target_dir, exist_ok=True)
        
        # Construct full target path
        target_path = os.path.join(target_dir, filename)
        
        # Only move if source and target paths are different
        if file_path != target_path:
            try:
                shutil.move(file_path, target_path)
                print(f"Moved: {filename} -> {target_dir_name}/")
            except Exception as e:
                print(f"Error moving {filename}: {str(e)}")


In [17]:
task = '''scans a folder called MyDownlads, identify file types and 
move them into categorized folders (e.g., PDFs, images, code files, etc)'''

In [20]:
prompt = f'''You are a AI assistant. Your job is to break a task into smaller 
 tasks that could be executed as code. Do not write the code, but just list the tasks as steps. Here is your task -
   {task}'''

In [21]:
response = model.generate_content(prompt)
print(response.text)

Okay, I can help break that task down into smaller steps:

1.  **Set up the Environment:**
    *   Import necessary Python libraries (e.g., `os`, `shutil`, `mimetypes`).
    *   Define the source folder (MyDownloads) path.
    *   Define a dictionary or data structure to map file extensions to category folder names (e.g., {".pdf": "PDFs", ".jpg": "Images", ".py": "Code"}).

2.  **Folder Structure Validation:**
    *   Check if the source folder (MyDownloads) exists. If not, handle the error (e.g., print an error message and exit or create the folder).
    *   Create destination folders for each file category (e.g., PDFs, Images, Code) within the same parent directory as the source folder, if they don't already exist.

3.  **File Iteration and Analysis:**
    *   Iterate through all files within the source folder.
    *   For each file:
        *   Extract the file extension.
        *   Determine the file type (e.g., using `mimetypes.guess_type` or by checking the file extension agains

In [22]:
# Write code to get a gemini model
import google.generativeai as genai
genai.configure(api_key="google_api_key")
model_getFunctions = genai.GenerativeModel('gemini-2.0-flash-exp')


In [23]:
functions = '''
def get_files_in_folder(root_dir):    
    """Recursively scan a folder and return list of files with full paths""" 
    
def get_file_types(file_list):    
    """Return dictionary mapping filenames to their file extensions."""

def organize_files(type_dict):
    """Organize files into folders based on their type in their respective directories."""
  
    
    
'''

In [28]:
prompt2 = f'''You are a AI assistant. You are given some steps. 
Your job is to get the function calls for these steps from the list of functions below. 
For each step give me the function that you would call along with parameters. 
You shall only use the functions mentioned here and you shall not write your own code. 
Here are your steps - {response.text} ; and
here are your functions - {functions}'''

In [29]:
response2 = model_getFunctions.generate_content(prompt2)

In [30]:
print(response2.text)

Okay, I understand the steps and the available functions. Here's the breakdown of function calls for each step:

**1. Set up the Environment:**

*   This step doesn't directly involve any of the provided functions. It's about setting up the environment within the main script (importing libraries, defining paths, and creating the extension-to-category mapping). No function call needed here.

**2. Folder Structure Validation:**

*   This step also doesn't directly involve any of the provided functions. It's about validating the source folder and creating destination folders. No function call needed here.

**3. File Iteration and Analysis:**

*   **Call:** `get_files_in_folder(root_dir)`
    *   **Parameter:** `root_dir` - This would be the path to the "MyDownloads" folder.
    *   **Purpose:** This function retrieves a list of all files within the source folder.
*   **Call:** `get_file_types(file_list)`
    *   **Parameter:** `file_list` - This would be the list of files returned by `get